In [1]:
from tsapix.envbase import *
from tsapix.utils import env_db_root

from tsapix.ibkr.qqq import *

In [2]:
app = DataApp()
app.connect("127.0.0.1", 4001, 1)
threading.Thread(target=app.run).start()
time.sleep(1)

reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:hfarm, orderReject: 
reqId: -1, errorCode: 2106, errorString: HMDS data farm connection is OK:apachmds, orderReject: 
reqId: -1, errorCode: 2158, errorString: Sec-def data farm connection is OK:secdefhk, orderReject: 


In [3]:
ltdlist = [f"{xdt} 16:00:00 US/Eastern" for xdt in [
    (datetime.datetime.today() - datetime.timedelta(x)).strftime("%Y%m%d") for x in range(1, 2)
]]
barsetting = [("1 D", "1 min", "TRADES")]
# barsetting = [("1 D", "5 mins", "TRADES")]

from itertools import product

_reqlist = list(product(ltdlist, barsetting))

paramslog = {}


In [4]:
_reqlist

[('20250914 16:00:00 US/Eastern', ('1 D', '1 min', 'TRADES'))]

In [5]:
from tqdm import tqdm

for _req in tqdm(_reqlist):
    _ltd, _bst = _req
    _ltddate = _ltd.split(" ")[0]
    _contract = init_qqq()

    _reqid = app.nextId()
    paramslog[_reqid] = [_ltd, _bst, _ltddate]
    print("Processing", _reqid, paramslog[_reqid])
    app.reqHistoricalData(
        _reqid, 
        _contract, 
        _ltd, 
        _bst[0], 
        _bst[1], 
        _bst[2], 
        1, 
        1, 
        False, 
        [])

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2122.62it/s]

Processing 2 ['20250914 16:00:00 US/Eastern', ('1 D', '1 min', 'TRADES'), '20250914']


reqId: -1, errorCode: 2106, errorString: HMDS data farm connection is OK:ushmds, orderReject: 
2 Date: 20250912 09:30:00 US/Eastern, Open: 585.04, High: 585.57, Low: 585.01, Close: 585.19, Volume: 579727, WAP: 585.299, BarCount: 3457
2 Date: 20250912 09:31:00 US/Eastern, Open: 585.21, High: 585.31, Low: 584.69, Close: 585.15, Volume: 295291, WAP: 585.036, BarCount: 2068
2 Date: 20250912 09:32:00 US/Eastern, Open: 585.13, High: 585.16, Low: 584.78, Close: 584.78, Volume: 215831, WAP: 584.984, BarCount: 1562
2 Date: 20250912 09:33:00 US/Eastern, Open: 584.8, High: 585.17, Low: 584.8, Close: 585.04, Volume: 147187, WAP: 584.998, BarCount: 1027
2 Date: 20250912 09:34:00 US/Eastern, Open: 585.05, High: 585.06, Low: 584.81, Close: 584.87, Volume: 121067, WAP: 584.929, BarCount: 815
2 Date: 20250912 09:35:00 US/Eastern, Open: 584.88, High: 584.88, Low: 584.5, Close: 584.57, Volume: 162867, WAP: 584.71, BarCount: 1164
2 Date: 20250912 09:36:00 US/Eastern, Open: 584.55, High: 584.81, Low: 584.2

In [6]:
def get_qqq_ohlcv_bardatapd(rawdatalist, _reqid, suffix=None, savepath=None, paramslog={}):
    import re
    import pandas as pd
    import os
    barsdata = re.findall(
    "([0-9]{2}[:][0-9]{2}[:][0-9]{2}) US/Eastern, Open: ([.0-9]+), High: ([.0-9]+), Low: ([.0-9]+), Close: ([.0-9]+), Volume: ([.0-9]+)", 
    " ".join([str(x) for x in rawdatalist]))
    _ltd, _bst, _ltddate = paramslog[_reqid]
    frq = _bst[1].replace(" ", "")
    fpath = f'{savepath}/'
    os.makedirs(fpath, exist_ok=True)
    if suffix is None:
        pd.DataFrame(barsdata).to_csv(f'{fpath}/{_ltddate}.csv')
    else:
        pd.DataFrame(barsdata).to_csv(f'{fpath}/{_ltddate}.csv')

In [7]:
strdatapool = {}
for k, v in app.datapool.items():
    strdatapool[k] = [str(x) for x in v]
    
    
save_root = env_db_root + "/QQQ/ohlcv2/"
os.makedirs(save_root, exist_ok=True)

for k, v in strdatapool.items():
    _suffix = None
    get_qqq_ohlcv_bardatapd(v, k, _suffix, 
                      savepath=save_root, paramslog=paramslog)

    

reqId: -1, errorCode: 2107, errorString: HMDS data farm connection is inactive but should be available upon demand.apachmds, orderReject: 
reqId: -1, errorCode: 2107, errorString: HMDS data farm connection is inactive but should be available upon demand.ushmds, orderReject: 


## remove invalid dates

In [ ]:
import yfinance as yf

qqqpx = yf.download('QQQ', )

In [ ]:
validdts = [x.strftime("%Y%m%d") for x in qqqpx.index]

In [ ]:
import os

ohlcvpath = "/Users/my/Library/Mobile Documents/com~apple~CloudDocs/env/projects/us_intraday/QQQ/ohlcv2/"
instockdts = [f for f in os.listdir(ohlcvpath) if ".csv" in f]

In [ ]:
removed = 0
for f in instockdts:
    if f.replace(".csv", "") not in validdts:
        os.remove(f"{ohlcvpath}/{f}")
        removed+=1

reqId: -1, errorCode: 2157, errorString: Sec-def data farm connection is broken:secdefhk, orderReject: 
reqId: -1, errorCode: 2103, errorString: Market data farm connection is broken:hfarm, orderReject: 
reqId: -1, errorCode: 2158, errorString: Sec-def data farm connection is OK:secdefhk, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:hfarm, orderReject: 
reqId: -1, errorCode: 1100, errorString: Connectivity between IBKR and Trader Workstation has been lost., orderReject: 
reqId: -1, errorCode: 2103, errorString: Market data farm connection is broken:hfarm, orderReject: 
reqId: -1, errorCode: 2157, errorString: Sec-def data farm connection is broken:secdefhk, orderReject: 
reqId: -1, errorCode: 2158, errorString: Sec-def data farm connection is OK:secdefhk, orderReject: 
reqId: -1, errorCode: 2104, errorString: Market data farm connection is OK:hfarm, orderReject: 
reqId: -1, errorCode: 1102, errorString: Connectivity between IBKR and Trader Wo